In [2]:
# Cell 1: Setup
import sys
import os

# Set working directory to project root
notebook_dir = os.getcwd()
if 'notebooks' in notebook_dir:
    project_root = os.path.dirname(notebook_dir)
    os.chdir(project_root)

print(f"Working directory: {os.getcwd()}")

# Now add to path
sys.path.append('.')

from app.database import MovieDB
from app.query_processor import parse_query
from app.llm_service import generate_response
import pandas as pd
import json

# Verify database exists
import os
db_path = "data/movies.db"
print(f"Database exists: {os.path.exists(db_path)}")
print(f"Database path: {os.path.abspath(db_path)}")

db = MovieDB()

Working directory: /Users/vikrambhat/Documents/movie-rag-api/movie-rag


ModuleNotFoundError: No module named 'app'

In [6]:
import os
os.chdir('..')  # Go up one level if in notebooks/
print(f"Working dir: {os.getcwd()}")

Working dir: /Users/vikrambhat/Documents/movie-rag-api


In [7]:
# Cell 2: Test Database Queries

print("=== Database Tests ===\n")

# Test 1: Search by title
movies = db.search(title="inception")
print(f"Search 'inception': {len(movies)} results")
if movies:
    print(f"  → {movies[0]['title']} ({movies[0]['year']})")

# Test 2: Search by genre
action = db.search(genre="action", limit=3)
print(f"\nAction movies: {len(action)} results")
for m in action:
    print(f"  → {m['title']} - {m['vote_average']}/10")

# Test 3: Search by year
movies_2015 = db.search(year=2015, limit=5)
print(f"\nMovies from 2015: {len(movies_2015)} results")

# Test 4: Combined filters
sci_fi_2015 = db.search(genre="science fiction", year=2015, limit=3)
print(f"\nSci-fi from 2015: {len(sci_fi_2015)} results")

# Test 5: Top rated
top = db.get_top_rated(limit=5)
print(f"\nTop rated movies:")
for m in top:
    print(f"  → {m['title']} - {m['vote_average']}/10")

=== Database Tests ===

DEBUG Query: 
            SELECT id, title, year, genres, overview, 
                vote_average, vote_count, movie_cast, director
            FROM movies 
            WHERE vote_average >= ?
         AND title LIKE ? ORDER BY vote_average DESC, vote_count DESC LIMIT ?
DEBUG Params: [0.0, '%inception%', 5]
Search 'inception': 1 results
  → Inception (2010.0)
DEBUG Query: 
            SELECT id, title, year, genres, overview, 
                vote_average, vote_count, movie_cast, director
            FROM movies 
            WHERE vote_average >= ?
         AND genres LIKE ? ORDER BY vote_average DESC, vote_count DESC LIMIT ?
DEBUG Params: [0.0, '%action%', 3]

Action movies: 3 results
  → The Dark Knight - 8.2/10
  → The Empire Strikes Back - 8.2/10
  → Seven Samurai - 8.2/10
DEBUG Query: 
            SELECT id, title, year, genres, overview, 
                vote_average, vote_count, movie_cast, director
            FROM movies 
            WHERE vote_average 

In [8]:
# Cell 3: Test Query Processor

print("=== Query Processor Tests ===\n")

test_queries = [
    "Recommend action movies from 2015",
    "Tell me about Inception",
    "What are the best movies?",
    "Show me comedy films",
    "Find sci-fi movies from 2010"
]

for query in test_queries:
    result = parse_query(query)
    print(f"Query: {query}")
    print(f"  Intent: {result['intent']}")
    print(f"  Genre: {result['genre']}")
    print(f"  Year: {result['year']}")
    print(f"  Keywords: {result['keywords']}")
    print()

=== Query Processor Tests ===

Query: Recommend action movies from 2015
  Intent: recommend
  Genre: action
  Year: 2015
  Keywords: None

Query: Tell me about Inception
  Intent: describe
  Genre: None
  Year: None
  Keywords: inception

Query: What are the best movies?
  Intent: top_rated
  Genre: None
  Year: None
  Keywords: are ?

Query: Show me comedy films
  Intent: search
  Genre: comedy
  Year: None
  Keywords: None

Query: Find sci-fi movies from 2010
  Intent: search
  Genre: science fiction
  Year: 2010
  Keywords: None



In [9]:
# Cell 4: Test LLM Service

print("=== LLM Service Tests ===\n")

# Get some movies
movies = db.search(genre="action", year=2015, limit=3)

question = "Recommend action movies from 2015"
answer = generate_response(question, movies, intent='recommend')

print(f"Question: {question}")
print(f"\nMovies retrieved: {len(movies)}")
for m in movies:
    print(f"  - {m['title']}")

print(f"\nLLM Answer:\n{answer}")

=== LLM Service Tests ===

DEBUG Query: 
            SELECT id, title, year, genres, overview, 
                vote_average, vote_count, movie_cast, director
            FROM movies 
            WHERE vote_average >= ?
         AND genres LIKE ? AND year = ? ORDER BY vote_average DESC, vote_count DESC LIMIT ?
DEBUG Params: [0.0, '%action%', 2015, 3]
Question: Recommend action movies from 2015

Movies retrieved: 3
  - Baahubali: The Beginning
  - Avengers: Age of Ultron
  - Furious 7

LLM Answer:
If you're looking for some high-octane action in 2015, I'd recommend checking out Furious 7 - it's an adrenaline-fueled ride with plenty of intense car chases and revenge plot twists. Another option could be Avengers: Age of Ultron, which brings together a star-studded cast to take on a powerful villain that threatens the world. Both movies have received great reviews for their action-packed sequences!


In [10]:
# Cell 5: End-to-End Pipeline Test

print("=== End-to-End Pipeline Tests ===\n")

def test_pipeline(query):
    print(f"Query: {query}")
    
    # Parse
    query_info = parse_query(query)
    print(f"  Parsed: {query_info}")
    
    # Search
    if query_info['intent'] == 'top_rated':
        movies = db.get_top_rated(limit=5)
    else:
        movies = db.search(
            title=query_info.get('keywords'),
            genre=query_info.get('genre'),
            year=query_info.get('year'),
            limit=5
        )
    print(f"  Found: {len(movies)} movies")
    
    # Generate
    answer = generate_response(query, movies, intent=query_info['intent'])
    print(f"  Answer: {answer[:150]}...")
    print()
    
    return movies, answer

# Test various queries
queries = [
    "Tell me about The Matrix",
    "Recommend comedy movies from 2010",
    "What are the best sci-fi films?"
]

for q in queries:
    test_pipeline(q)

=== End-to-End Pipeline Tests ===

Query: Tell me about The Matrix
  Parsed: {'intent': 'describe', 'genre': None, 'year': None, 'keywords': 'matrix'}
DEBUG Query: 
            SELECT id, title, year, genres, overview, 
                vote_average, vote_count, movie_cast, director
            FROM movies 
            WHERE vote_average >= ?
         AND title LIKE ? ORDER BY vote_average DESC, vote_count DESC LIMIT ?
DEBUG Params: [0.0, '%matrix%', 5]
  Found: 3 movies
  Answer: The Matrix is an iconic science fiction film released in 1999 that follows the story of a computer hacker who joins a group of rebels fighting against...

Query: Recommend comedy movies from 2010
  Parsed: {'intent': 'recommend', 'genre': 'comedy', 'year': 2010, 'keywords': None}
DEBUG Query: 
            SELECT id, title, year, genres, overview, 
                vote_average, vote_count, movie_cast, director
            FROM movies 
            WHERE vote_average >= ?
         AND genres LIKE ? AND year = ? O

In [11]:
# Cell 6: Evaluation Dataset

print("=== Create Evaluation Dataset ===\n")

eval_queries = [
    {
        "query": "Recommend action movies from 2015",
        "expected_genre": "action",
        "expected_year": 2015,
        "expected_intent": "recommend"
    },
    {
        "query": "Tell me about Inception",
        "expected_keywords": "inception",
        "expected_intent": "describe"
    },
    {
        "query": "What are the best movies?",
        "expected_intent": "top_rated"
    },
    {
        "query": "Show me sci-fi films",
        "expected_genre": "science fiction",
        "expected_intent": "search"
    }
]

results = []

for test in eval_queries:
    query = test['query']
    parsed = parse_query(query)
    
    # Check intent
    intent_match = parsed['intent'] == test.get('expected_intent')
    
    # Check genre
    genre_match = True
    if 'expected_genre' in test:
        genre_match = parsed['genre'] == test['expected_genre']
    
    # Check year
    year_match = True
    if 'expected_year' in test:
        year_match = parsed['year'] == test['expected_year']
    
    results.append({
        'query': query,
        'intent_correct': intent_match,
        'genre_correct': genre_match,
        'year_correct': year_match,
        'all_correct': intent_match and genre_match and year_match
    })

df = pd.DataFrame(results)
print(df)
print(f"\nAccuracy: {df['all_correct'].sum()}/{len(df)} = {df['all_correct'].mean():.1%}")

=== Create Evaluation Dataset ===

                               query  intent_correct  genre_correct  \
0  Recommend action movies from 2015            True           True   
1            Tell me about Inception            True           True   
2          What are the best movies?            True           True   
3               Show me sci-fi films            True           True   

   year_correct  all_correct  
0          True         True  
1          True         True  
2          True         True  
3          True         True  

Accuracy: 4/4 = 100.0%


In [12]:
# Cell 7: Retrieval Quality Evaluation

print("=== Retrieval Quality Evaluation ===\n")

# Manual ground truth
ground_truth = {
    "inception": [27205],  # Movie IDs that should be returned
    "the matrix": [603],
    "action 2015": [76341, 102899, 177677]  # Any of these
}

def evaluate_retrieval(query, expected_ids):
    parsed = parse_query(query)
    movies = db.search(
        title=parsed.get('keywords'),
        genre=parsed.get('genre'),
        year=parsed.get('year'),
        limit=5
    )
    
    retrieved_ids = [m['id'] for m in movies]
    
    # Check if any expected ID is in retrieved
    hit = any(eid in retrieved_ids for eid in expected_ids)
    
    return {
        'query': query,
        'retrieved': len(movies),
        'hit': hit,
        'top_result': movies[0]['title'] if movies else None
    }

retrieval_results = []
for query, expected_ids in ground_truth.items():
    result = evaluate_retrieval(query, expected_ids)
    retrieval_results.append(result)
    print(f"{query}: {'✓' if result['hit'] else '✗'} - {result['top_result']}")

retrieval_df = pd.DataFrame(retrieval_results)
print(f"\nRetrieval Hit Rate: {retrieval_df['hit'].mean():.1%}")

=== Retrieval Quality Evaluation ===

DEBUG Query: 
            SELECT id, title, year, genres, overview, 
                vote_average, vote_count, movie_cast, director
            FROM movies 
            WHERE vote_average >= ?
         AND title LIKE ? ORDER BY vote_average DESC, vote_count DESC LIMIT ?
DEBUG Params: [0.0, '%inception%', 5]
inception: ✓ - Inception
DEBUG Query: 
            SELECT id, title, year, genres, overview, 
                vote_average, vote_count, movie_cast, director
            FROM movies 
            WHERE vote_average >= ?
         AND title LIKE ? ORDER BY vote_average DESC, vote_count DESC LIMIT ?
DEBUG Params: [0.0, '%matrix%', 5]
the matrix: ✓ - The Matrix
DEBUG Query: 
            SELECT id, title, year, genres, overview, 
                vote_average, vote_count, movie_cast, director
            FROM movies 
            WHERE vote_average >= ?
         AND genres LIKE ? AND year = ? ORDER BY vote_average DESC, vote_count DESC LIMIT ?
DEBUG Para

In [13]:
# Cell 8: Response Quality Check

print("=== Response Quality Check ===\n")

def check_response_quality(query, movies):
    answer = generate_response(query, movies)
    
    # Basic checks
    checks = {
        'has_movie_title': any(m['title'].lower() in answer.lower() for m in movies),
        'mentions_genre': any(g.lower() in answer.lower() for m in movies for g in m.get('genres', [])),
        'has_rating': any(str(m['vote_average']) in answer for m in movies),
        'length_ok': 50 < len(answer) < 500
    }
    
    return checks

# Test on a few queries
test_cases = [
    ("Recommend action movies from 2015", db.search(genre="action", year=2015, limit=3)),
    ("Tell me about The Matrix", db.search(title="matrix", limit=1))
]

quality_results = []
for query, movies in test_cases:
    if not movies:
        continue
    
    checks = check_response_quality(query, movies)
    checks['query'] = query
    quality_results.append(checks)
    
    print(f"\nQuery: {query}")
    for check, passed in checks.items():
        if check != 'query':
            print(f"  {check}: {'✓' if passed else '✗'}")

quality_df = pd.DataFrame(quality_results)
print("\n=== Quality Summary ===")
for col in quality_df.columns:
    if col != 'query':
        print(f"{col}: {quality_df[col].mean():.1%}")

=== Response Quality Check ===

DEBUG Query: 
            SELECT id, title, year, genres, overview, 
                vote_average, vote_count, movie_cast, director
            FROM movies 
            WHERE vote_average >= ?
         AND genres LIKE ? AND year = ? ORDER BY vote_average DESC, vote_count DESC LIMIT ?
DEBUG Params: [0.0, '%action%', 2015, 3]
DEBUG Query: 
            SELECT id, title, year, genres, overview, 
                vote_average, vote_count, movie_cast, director
            FROM movies 
            WHERE vote_average >= ?
         AND title LIKE ? ORDER BY vote_average DESC, vote_count DESC LIMIT ?
DEBUG Params: [0.0, '%matrix%', 1]

Query: Recommend action movies from 2015
  has_movie_title: ✓
  mentions_genre: ✓
  has_rating: ✗
  length_ok: ✓

Query: Tell me about The Matrix
  has_movie_title: ✓
  mentions_genre: ✓
  has_rating: ✓
  length_ok: ✓

=== Quality Summary ===
has_movie_title: 100.0%
mentions_genre: 100.0%
has_rating: 50.0%
length_ok: 100.0%


In [14]:
# Cell 9: Latency Benchmarking

import time

print("=== Latency Benchmarking ===\n")

def benchmark_query(query, runs=3):
    times = {
        'parse': [],
        'search': [],
        'llm': [],
        'total': []
    }
    
    for _ in range(runs):
        start = time.time()
        
        # Parse
        t1 = time.time()
        parsed = parse_query(query)
        times['parse'].append(time.time() - t1)
        
        # Search
        t2 = time.time()
        movies = db.search(
            title=parsed.get('keywords'),
            genre=parsed.get('genre'),
            year=parsed.get('year')
        )
        times['search'].append(time.time() - t2)
        
        # LLM
        t3 = time.time()
        answer = generate_response(query, movies, parsed['intent'])
        times['llm'].append(time.time() - t3)
        
        times['total'].append(time.time() - start)
    
    # Calculate averages
    avg_times = {k: sum(v)/len(v) for k, v in times.items()}
    return avg_times

# Benchmark
query = "Recommend action movies from 2015"
results = benchmark_query(query, runs=3)

print(f"Query: {query}\n")
print(f"Parse:     {results['parse']*1000:.1f}ms")
print(f"Search:    {results['search']*1000:.1f}ms")
print(f"LLM:       {results['llm']*1000:.1f}ms")
print(f"Total:     {results['total']*1000:.1f}ms")

=== Latency Benchmarking ===

DEBUG Query: 
            SELECT id, title, year, genres, overview, 
                vote_average, vote_count, movie_cast, director
            FROM movies 
            WHERE vote_average >= ?
         AND genres LIKE ? AND year = ? ORDER BY vote_average DESC, vote_count DESC LIMIT ?
DEBUG Params: [0.0, '%action%', 2015, 5]
DEBUG Query: 
            SELECT id, title, year, genres, overview, 
                vote_average, vote_count, movie_cast, director
            FROM movies 
            WHERE vote_average >= ?
         AND genres LIKE ? AND year = ? ORDER BY vote_average DESC, vote_count DESC LIMIT ?
DEBUG Params: [0.0, '%action%', 2015, 5]
DEBUG Query: 
            SELECT id, title, year, genres, overview, 
                vote_average, vote_count, movie_cast, director
            FROM movies 
            WHERE vote_average >= ?
         AND genres LIKE ? AND year = ? ORDER BY vote_average DESC, vote_count DESC LIMIT ?
DEBUG Params: [0.0, '%action%', 2

In [16]:
# Cell 10: Export Results

print("=== Export Evaluation Results ===\n")

# Combine all results
evaluation_report = {
    'query_parser_accuracy': df['all_correct'].mean(),
    'retrieval_hit_rate': retrieval_df['hit'].mean(),
    'response_quality': {
        col: quality_df[col].mean() 
        for col in quality_df.columns if col != 'query'
    },
    'latency_ms': {
        k: v*1000 for k, v in results.items()
    }
}

# Save to JSON
with open('../evaluation_results.json', 'w') as f:
    json.dump(evaluation_report, f, indent=2)

print("Results saved to evaluation_results.json")
print("\n=== Summary ===")
print(json.dumps(evaluation_report, indent=2))


=== Export Evaluation Results ===

Results saved to evaluation_results.json

=== Summary ===
{
  "query_parser_accuracy": 1.0,
  "retrieval_hit_rate": 1.0,
  "response_quality": {
    "has_movie_title": 1.0,
    "mentions_genre": 1.0,
    "has_rating": 0.5,
    "length_ok": 1.0
  },
  "latency_ms": {
    "parse": 0.03631909688313802,
    "search": 2.385218938191732,
    "llm": 1441.6797161102295,
    "total": 1444.1036383310955
  }
}


In [1]:
"""
SQL Agent implementation using LangChain
Alternative approach demonstrating autonomous query generation
"""

from langchain_ollama import ChatOllama
from langchain_community.agent_toolkits import create_sql_agent
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from typing import Dict
import logging

from app.config import DATABASE_PATH, OLLAMA_MODEL

logger = logging.getLogger(__name__)

# Initialize database connection
db_uri = f"sqlite:///{DATABASE_PATH}"
db = SQLDatabase.from_uri(db_uri)

# Initialize LLM
llm = ChatOllama(
    model=OLLAMA_MODEL,
    temperature=0 
)

# Create toolkit
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

# Create agent with safety constraints
agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=False,
    agent_type="tool-calling",
    handle_parsing_errors=True,
    max_iterations=3,  
    max_execution_time=10  
)


def query_with_agent(question: str) -> Dict:
    """
    Use SQL Agent for autonomous query generation
    
    Agent can generate and execute SQL queries based on natural language.
    More flexible but less predictable than traditional approach.
    """
    try:
        logger.info(f"Agent processing: {question}")
        
        updated_question = (
            "Answer this question concisely in 1-2 sentences. "
            "Do not include SQL queries, tool calls, or reasoning steps in your answer. "
            "Only provide the final answer.\n\n"
            f"Question: {question}"
        )
        # Run agent
        result = agent.invoke({"input": updated_question})
        
        # Extract answer
        answer = result.get('output', 'No response generated')
        
        return {
            "answer": answer,
            "method": "sql_agent",
            "note": "Response generated via autonomous SQL agent"
        }
        
    except Exception as e:
        logger.error(f"Agent error: {e}")
        return {
            "answer": f"Agent encountered an error. Please try rephrasing your question.",
            "method": "agent_error",
            "error": str(e)
        }


def get_agent_info() -> Dict:
    """Get information about available database tables and schema"""
    try:
        table_info = db.get_table_info()
        return {
            "tables": db.get_usable_table_names(),
            "schema": table_info
        }
    except Exception as e:
        return {"error": str(e)}

ModuleNotFoundError: No module named 'app'

In [23]:
# Cell 12: Agent Capability Testing

print("=== Agent Capability Tests ===\n")

# Test complex queries that traditional approach can't handle
complex_queries = [
    "Which year had the most movies released?",
    "What's the average rating of action movies?",
    "List directors with more than 3 movies",
    "How many movies have rating above 8.5?"
]

agent_results = []

for query in complex_queries:
    print(f"\nQuery: {query}")
    result = query_with_agent(query)
    
    print(f"Answer: {result['answer']}")
    print(f"Method: {result['method']}")
    
    agent_results.append({
        'query': query,
        'success': result['method'] != 'agent_error',
        'answer_length': len(result['answer'])
    })

agent_df = pd.DataFrame(agent_results)
print(f"\n=== Agent Success Rate ===")
print(f"Success: {agent_df['success'].sum()}/{len(agent_df)} = {agent_df['success'].mean():.1%}")

Agent error: Client.chat() got an unexpected keyword argument 'think'
Agent error: Client.chat() got an unexpected keyword argument 'think'
Agent error: Client.chat() got an unexpected keyword argument 'think'
Agent error: Client.chat() got an unexpected keyword argument 'think'


=== Agent Capability Tests ===


Query: Which year had the most movies released?
Answer: Agent encountered an error. Please try rephrasing your question.
Method: agent_error

Query: What's the average rating of action movies?
Answer: Agent encountered an error. Please try rephrasing your question.
Method: agent_error

Query: List directors with more than 3 movies
Answer: Agent encountered an error. Please try rephrasing your question.
Method: agent_error

Query: How many movies have rating above 8.5?
Answer: Agent encountered an error. Please try rephrasing your question.
Method: agent_error

=== Agent Success Rate ===
Success: 0/4 = 0.0%


In [27]:
# Cell 13: Agent Robustness Testing

print("=== Agent Robustness Tests ===\n")

# Edge cases
edge_cases = [
    "Show me movies from year 3000",  # Impossible query
    "What is the meaning of life?",   # Non-movie query
    "Delete all movies",              # Dangerous query (should be blocked)
    "",                               # Empty query
]

for query in edge_cases:
    print(f"\nQuery: '{query}'")
    result = query_with_agent(query)
    
    if result['method'] == 'agent_error':
        print(f"  ✓ Handled gracefully: {result['answer']}")
    else:
        print(f"  Answer: {result['answer'][:80]}...")

Agent error: Client.chat() got an unexpected keyword argument 'think'
Agent error: Client.chat() got an unexpected keyword argument 'think'
Agent error: Client.chat() got an unexpected keyword argument 'think'
Agent error: Client.chat() got an unexpected keyword argument 'think'


=== Agent Robustness Tests ===


Query: 'Show me movies from year 3000'
  ✓ Handled gracefully: Agent encountered an error. Please try rephrasing your question.

Query: 'What is the meaning of life?'
  ✓ Handled gracefully: Agent encountered an error. Please try rephrasing your question.

Query: 'Delete all movies'
  ✓ Handled gracefully: Agent encountered an error. Please try rephrasing your question.

Query: ''
  ✓ Handled gracefully: Agent encountered an error. Please try rephrasing your question.


In [26]:
pip install langchain-ollama==0.2.2

  Obtaining dependency information for langchain-ollama==0.2.2 from https://files.pythonhosted.org/packages/7f/77/219fb2290c832e33af2731246ea3328bade50756288c1e97ae73c4ccc197/langchain_ollama-0.2.2-py3-none-any.whl.metadata
  Using cached langchain_ollama-0.2.2-py3-none-any.whl.metadata (1.9 kB)
  Obtaining dependency information for langchain-core<0.4.0,>=0.3.27 from https://files.pythonhosted.org/packages/fc/71/46b0efaf3fc6ad2c2bd600aef500f1cb2b7038a4042f58905805630dd29d/langchain_core-0.3.79-py3-none-any.whl.metadata
  Using cached langchain_core-0.3.79-py3-none-any.whl.metadata (3.2 kB)
Using cached langchain_ollama-0.2.2-py3-none-any.whl (18 kB)
Using cached langchain_core-0.3.79-py3-none-any.whl (449 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.0.0
    Uninstalling langchain-core-1.0.0:
      Successfully uninstalled langchain-core-1.0.0
  Attempting uninstall: langchain-ollama
    Found existing installation: langchain-ollama 1.0.0